In [ ]:
# pour telecharger la dataset, il faut demander un lien unique
# a travers cette forme: https://cornell.qualtrics.com/jfe/form/SV_6YA3HQ2p75XH4IR

In [ ]:
# quelques statistiques faits
----- First 100k articles and summaries from dev: -----
Avg. article length (words) 551.2604
Avg. summary length (words):  33.9256
Avg. article length (sentences) 32.3397
Avg. summary length (sentences):  1.8956

In [ ]:
! pip install wget datasets transformers rouge-score textacy wandb

In [ ]:
import wandb
import datetime

In [ ]:
wandb_key = # code d'acces pour enregsitrer les graphes
hf_token = # code d'acces pour uploader le modele

In [ ]:
wandb.login(key=wandb_key)

now = datetime.datetime.now()
proj = "big  modelof" + str(now.hour) +'-'+ str(now.minute)
# Initialize a wandb run
wandb.init(project=proj)

In [ ]:
import os
from wget import download

if "release" not in os.listdir():
    
    #downloading dataset
    if "newsroom-release.tar" not in os.listdir():
        print("* Downloading Dataset")
        download("https://lil.nlp.cornell.edu/resources/newsroom/r8625bda324/newsroom-release.tar")
    
    #unzipping tar file
    print("\n* Unzipping dataset")
    os.system("tar xvf newsroom-release.tar")
#unzipping gz archives
if "test.jsonl" not in os.listdir("release"):
    print("\n* Unzipping release files")
    os.system("gzip -d release/test.jsonl.gz")
    os.system("gzip -d release/train.jsonl.gz")
    os.system("gzip -d release/dev.jsonl.gz")
    print("\n++ Done")

In [ ]:
os.environ['WANDB_SILENT']="true" # turn off wandb warnings

In [ ]:
!ls release

In [ ]:
!rm  newsroom-release.tar

In [ ]:
def split_by_density(df_abs,df_mix,df_ext,r_abs,r_mix,r_ext): # r_abs ratio for abstractive split
    # cette fonction prend 3 splits et les fractions a donner a 
    # chaque split et les repartis en fonction de ces fractions
    # par exemple 80% abstractif + 20% extractif + 30% mixte
    n_abs = len(df_abs);n_mix = len(df_mix);n_ext = len(df_ext)
    n_total = int(n_abs / r_abs) # we use # of abstractive rows as reference
    #fractions to take from each
    frac_mix = r_mix * n_abs /(r_abs * n_mix)
    frac_ext = r_ext * n_abs /(r_abs * n_ext)
    #take needed rows(randomly)
    ret_mix = df_mix.sample(frac = frac_mix)
    ret_ext = df_ext.sample(frac = frac_ext)
    #concatenate&return
    ret_df = pd.concat([df_abs,ret_mix,ret_ext])
    return ret_df

In [ ]:
import pandas as pd

#split settings
train_size = 300000
split_abs = 0.5
split_mix = 0.3
split_ext = 0.2

df_train = pd.read_json(r'release/train.jsonl', lines=True, nrows=train_size,keep_default_dates=False)

df_abs = df_train.loc[df_train["density_bin"]=="abstractive"]
df_mix = df_train.loc[df_train["density_bin"]=="mixed"]
df_ext = df_train.loc[df_train["density_bin"]=="extractive"]
#randomise and get splits 
df_train_final = split_by_density(df_abs,df_mix,df_ext,split_abs,split_mix,split_ext)


n = len(df_train_final)

#80% train 20% validation
dev_size = int((n/0.8) * 0.2)
df_dev = pd.read_json(r'release/dev.jsonl', lines=True, nrows=dev_size,keep_default_dates=False)


In [ ]:
print("ratio abstractive: ",len(df_train_final[df_train_final["density_bin"]=="abstractive"])/n)
print("ratio mixed: ",len(df_train_final[df_train_final["density_bin"]=="mixed"])/n)
print("ratio extractive: ",len(df_train_final[df_train_final["density_bin"]=="extractive"])/n)

In [ ]:
# keep only needed columns (text & summary) to fasten training
df_train_final = df_train_final[['text', 'summary']]
df_dev = df_dev[['text', 'summary']]

In [ ]:
len(df_dev)/len(df_train_final)

In [ ]:
import torch
torch.cuda.is_available()
# verification si les gpus sont detectes

In [ ]:
#adding index to help with training
df_train_final.reset_index(inplace=True)
df_dev.reset_index(inplace=True)

#creation de l objet datasets
from datasets import Dataset,load_metric
raw_train_ds = Dataset.from_pandas(df_train_final)
raw_dev_ds = Dataset.from_pandas(df_dev)

metric = load_metric("rouge")

In [ ]:
raw_train_ds

In [ ]:
from datasets import concatenate_datasets
from datasets import DatasetDict


# Create a DatasetDict object that contains the two datasets
dataset = DatasetDict({
    "train": raw_train_ds,
    "validation": raw_dev_ds,
})

In [ ]:
dataset["validation"].features

In [ ]:
from transformers import BartForConditionalGeneration,BartTokenizerFast,DataCollatorForSeq2Seq,Seq2SeqTrainingArguments,Seq2SeqTrainer

# chargement de modele a entrainer
model_checkpoint = "facebook/bart-base"

model = BartForConditionalGeneration.from_pretrained(model_checkpoint)#,return_dict=True)
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoint)

In [ ]:
# pre traitement
from textacy import preprocessing
from functools import partial

In [ ]:
preproc = preprocessing.make_pipeline( # cleaning pipeline
  partial(preprocessing.replace.urls, repl=""),
  preprocessing.normalize.bullet_points,
  preprocessing.normalize.hyphenated_words,
  preprocessing.normalize.quotation_marks,
  preprocessing.normalize.unicode,
  preprocessing.normalize.whitespace,
  preprocessing.remove.accents,
  preprocessing.remove.brackets,
  preprocessing.remove.html_tags,
  partial(preprocessing.replace.emails, repl=""),
  partial(preprocessing.replace.emojis, repl=""),
  partial(preprocessing.replace.hashtags, repl=""),
  )

def batch_clean(examples):
  for i in range(len(examples)):
    examples["text"][i] = preproc(examples["text"][i])
    examples["summary"][i] = preproc(examples["summary"][i])
  return examples


In [ ]:
max_input_length = 1024
max_target_length = 128

#fonction qui tokenize le jeu de donnees

def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
#pretraitement
dataset["train"] = dataset["train"].map(batch_clean,batched=True)
dataset["validation"] = dataset["validation"].map(batch_clean,batched=True)

In [ ]:
dataset['train'][199]["summary"]

In [ ]:
#tokenization
tokenized_train_ds = dataset["train"].map(preprocess_function, batched=True)
tokenized_dev_ds = dataset["validation"].map(preprocess_function, batched=True)

In [ ]:
#specifique à colab;
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
def better_join(sent_list,sep = '\n'):
  n = len(sent_list)
  i = 0
  text = ''
  while i < n-1:
      text = text + str(sent_list[i]) + sep
      i = i+1
  return text

In [ ]:
import spacy
import numpy as np
nlp = spacy.load("en_core_web_sm")

# fonction utilisée en evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # tenseur de tokens -> texte
    # on efface -100 dans les labels vu qu on pas les decoder (element non trouvé dans le corpus)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) # tenseur de tokens -> texte
    
    # structuration du texte: rouge necessite \n entre les phrases
    decoded_preds = [better_join(list(nlp(pred.strip()).sents)) for pred in decoded_preds]
    decoded_labels = [better_join(list(nlp(label.strip()).sents)) for label in decoded_labels]
    # calcul de rouge
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # on extrait les resultats les plus importants (fmeasure du mid de chaque rouge)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # calcul de la longueur du texte generé
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# exemple de rouge

decoded_labels = ['hello mister.I am very healthy.Are you very healty? Thank you very much',
                 'Hello mister.I am very healthy.Are you very healty?',
                 'Hello mister.I am very healthy.Are you very healty?Thank you very much. I really like it']
                 
decoded_preds = ['hello.I am good.Are you? Thanks',
                  'hello.I am good.Are you?',
                  'hello.I am good.Are you?Thanks.I like it']


import pprint
def pretty(t):
  pp = pprint.PrettyPrinter(width=100,indent=2)
  pp.pprint(t)


decoded_preds = [better_join(list(nlp(pred.strip()).sents)) for pred in decoded_preds]
decoded_labels = [better_join(list(nlp(label.strip()).sents)) for label in decoded_labels]


pretty(metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True))

In [ ]:
# batching
batch_size = 10

#hyperparametres d entrainement
args = Seq2SeqTrainingArguments(
    output_dir='./mymodel_v2_2', # non du modele
    save_strategy = "epoch",
    report_to="wandb", # wandb pour l enregistrement des donnees et creation des graphes
    hub_strategy = 'checkpoint', # uploads every save_strategy
    evaluation_strategy = "steps",
    eval_steps = 4000, # evaluation chaque 4k etapes
    learning_rate=2e-4,
    optim="adamw_torch",
    per_device_train_batch_size=batch_size, # activation du batching sur les 2 gpus
    per_device_eval_batch_size=batch_size,  # activation du batching sur les 2 gpus
    weight_decay=0.01, # le weight decay (on utilise adamw pas adam)
    num_train_epochs=2,
    fp16=True, # nombre flottants = 16bits
    predict_with_generate=True, # pour generer un texte durant la validation
    push_to_hub = True, # on upload a la fin
    hub_token = hf_token
)


In [ ]:
!ls mymodel_v2_2

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model) # collateur des donnees
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_dev_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
!nvidia-smi

In [ ]:
!ls mymodel_v2_2

In [ ]:
result = trainer.train(resume_from_checkpoint = True)

In [ ]:
#torch.cuda.empty_cache() #vider memoire gpu

In [ ]:
print(result)

In [ ]:
trainer.save_model()

In [ ]:
predictions, labels, metrics = trainer.predict(raw_dev_ds)

In [ ]:
raw_dev_ds

In [ ]:
print(metrics)

In [ ]:
# test du modele
ARTICLE = """
Former NBCUniversal advertising chief Linda Yaccarino will become Twitter’s new CEO, current chief executive Elon Musk says in a tweet, as the social media platform seeks to reverse a plunge in advertisement revenues.
“I am excited to welcome Linda Yaccarino as the new CEO of Twitter!” Musk said on Friday. “@LindaYacc will focus primarily on business operations, while I focus on product design & new technology.”
Yaccarino, who modernised the advertising business at the Comcast Corp entertainment and media division, will have her hands full as she takes on a firm loaded with debt, among other challenges.
Since Musk acquired Twitter in October, advertisers have fled the platform, worried that their ads could appear next to inappropriate content after the company lost nearly 80 percent of its staff. Musk this year acknowledged that Twitter suffered a massive decline in advertisement revenue.
Musk has axed thousands of employees, rushed the launch of a subscription product that allowed scammers to impersonate major brands and suspended users with whom he disagreed.
Twitter’s “trajectory will immediately take a 180-degree turn” under Yaccarino’s leadership, said Lou Paskalis, a longtime advertisement industry executive and CEO of AJL Advisory, a marketing consultancy.
“I think [Yaccarino] has climbed every mountain she could at NBCU and did it impeccably well, and there’s no greater challenge than restoring order at Twitter,” he said.
Yaccarino could not be reached for comment.
While Musk said Yaccarino would help build an “everything app”, which he has previously said could offer a variety of services such as peer-to-peer payments, his selection of an advertising veteran signalled that digital ads would continue to be a core focus of the business.
In order to diversify revenue sources, Musk has focused on Twitter Blue, a subscription feature that costs users $8 per month to verify their accounts, but the product has had limited success.
Independent researcher Travis Brown, who has been tracking the number of Twitter Blue subscribers over time, estimated there were 619,858 customers as of April 30.
Yaccarino’s exit is another big blow to NBCUniversal. Last month, NBC parent Comcast said NBCUniversal CEO Jeff Shell was leaving after acknowledging an inappropriate relationship with a woman in the company after a complaint that prompted an investigation.
Advertising President Mark Marshall will step in as interim chairman of NBCUniversal’s advertising and partnerships group.
Yaccarino joined NBCU in 2011 after 15 years at Turner Entertainment and has been credited with taking the network’s advertisement sales operation into the digital era.
As broadcast television audiences migrated to streaming, she took to the stage at Radio City Music Hall last year to tell advertisers their brand messages were not an afterthought. She said NBCUniversal incorporated advertisements in its Peacock streaming service from the outset.
“Twitter needs credibility with the advertising community,” said Greg Kahn, chief executive of GK Digital Ventures media consultancy. “Linda has demonstrated her trust, her innovative nature of bringing new partners to the table and a deep bench of relationships.”
Musk, the CEO of electric vehicle maker Tesla, completed his purchase of Twitter in October for $44bn. He said in December that he would step aside as CEO once he found “someone foolish enough to take the job”.
On Thursday, Musk tweeted that he had found a CEO without naming Yaccarino. One person close to Yaccarino said Musk’s tweet may well have accelerated the timetable for her to join Twitter, which would be a balm to Tesla shareholders.
Shares of Tesla were down 1.3 percent on Friday even as analysts commented that a CEO hire would allow Musk to concentrate on the electric car business. Comcast shares were little changed.
"""

In [ ]:
len(tokenizer.encode(ARTICLE))

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="./mymodel_v2_2")


In [ ]:
print(summarizer(ARTICLE, max_length=128, do_sample=True)[0]["summary_text"])